<h2> Imports & Configuration </h2>

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import time

In [2]:
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [3]:
import cml.data_v1 as cmldata

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
#SparkContext.setSystemProperty('spark.executor.cores', '1')
#SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "paul-aug26-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

import os
print("https://spark-"+os.environ["CDSW_ENGINE_ID"]+"."+os.environ["CDSW_DOMAIN"])

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


Spark Application Id:spark-f9cadbf7ddbf4f69a8f5c073cf50d86d
https://spark-6pwfcs20zgmqc8bx.ml-f0942348-224.paul-aug.a465-9q4k.cloudera.site


In [4]:
# spark.conf.set("spark.sql.shuffle.partitions", "3")
spark.conf.set("spark.sql.adaptive.enabled", "false")

# Join Skews

In [5]:
STORAGE = "s3a://paul-aug26-buk-a3c2b50a/data/pdefusco"

#s3a://go01-demo/datalake/pdefusco/transactions/

transactions_file = "/transactions/*"
customer_file = "/pii/piiData.csv"

df_transactions = spark.read.parquet(STORAGE + transactions_file)
df_customers = spark.read.option("header",True).csv(STORAGE + customer_file)

In [6]:
#print(df_transactions.count())
#print(df_customers.count())

In [7]:
df_transactions.printSchema()
#df_transactions.show(5, False)

root
 |-- credit_card_number: string (nullable = true)
 |-- credit_card_provider: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- event_ts: timestamp (nullable = true)
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- transaction_currency: string (nullable = true)
 |-- transaction_amount: decimal(10,0) (nullable = true)



In [8]:
df_customers.printSchema()
#df_customers.show(5, False)

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- email: string (nullable = true)
 |-- aba_routing: string (nullable = true)
 |-- bank_country: string (nullable = true)
 |-- account_no: string (nullable = true)
 |-- int_account_no: string (nullable = true)
 |-- swift11: string (nullable = true)
 |-- credit_card_number: string (nullable = true)



In [9]:
#df_transactions.groupBy("credit_card_number").count().orderBy("credit_card_number").show()

In [10]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [11]:
df_txn_details = (
    df_transactions.join(
        df_customers,
        on="credit_card_number",
        how="inner"
    )
)

In [12]:
start_time = time.time()
df_txn_details.count()
print(f"time taken: {time.time() - start_time}")

time taken: 15.8869948387146


# Using AQE

In [13]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewedJoin.enabled", "true")

In [14]:
df_txn_details = (
    df_transactions.join(
        df_customers,
        on="credit_card_number",
        how="inner"
    )
)

In [15]:
start_time = time.time()
df_txn_details.count()
print(f"time taken: {time.time() - start_time}")

time taken: 9.830567836761475


# Using Broadcast Joins

In [16]:
# 10MB = 10485760 Bytes
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10485760)

In [17]:
df_txn_details = (
    df_transactions.join(
        F.broadcast(df_customers),
        on="credit_card_number",
        how="inner"
    )
)

In [18]:
start_time = time.time()
df_txn_details.count()
print(f"time taken: {time.time() - start_time}")

time taken: 9.444070100784302


In [19]:
# spark.stop()